In [2]:
import geopandas as gpd
import pandas as pd
import os
import time
import sv_config as sc

startTime = time.time()
dirname = os.path.abspath('')
gpkg_input = os.path.join(dirname, sc.output_hex_250m)
gdf_hex = gpd.read_file(gpkg_input, layer="odense")

# join pop_est from original hex to processed hex
gpkg_origin_hex = os.path.join(dirname, sc.geopackagePath)
gdf_hex_origin = gpd.read_file(gpkg_origin_hex, layer=sc.hex250)
gdf_study_region = gpd.read_file(gpkg_origin_hex, layer=sc.urban_study_region)


In [3]:
gdf_hex = gdf_hex.join(gdf_hex_origin.set_index('index'),on='index',how = 'left',rsuffix='_origin')

In [4]:
# import pandas as pd
# gdf1=pd.merge(gdf_hex,gdf_hex_origin,on='index',how='left',suffixes=('','_origin'))

In [7]:
gdf_study_region

,Study region,geometry,urban_sample_point_count
0,Odense,"MULTIPOLYGON (((589115.578 6145014.850, 589126...",50707


In [6]:
gdf_study_region['urban_sample_point_count'] = gdf_hex['urban_sample_point_count'].sum()

In [10]:
(gdf_hex['pop_est']*gdf_hex['pct_access_500m_supermarkets']).sum()/(gdf_hex['pop_est'].sum())

36.87057168378925

In [11]:
def aggregation(input_gdf, out_gdf, fieldNames):
    """create aggregation result for city from hexes
    Arguments:
        input_gdf {geodataframe} -- [hex]
        out_gdf {geodataframe} -- [city]
        fieldNames {list} -- [zipped input and output fieldnames]
    """
    for field in fieldNames:
        out_gdf[field[1]] = (input_gdf['pop_est'] * input_gdf[field[0]]
                             ).sum() / (input_gdf['pop_est'].sum())
    return out_gdf

In [12]:
fieldNames = [
    'pct_access_500m_supermarkets', 'pct_access_500m_convenience',
    'pct_access_500m_pt_any', 'pct_access_500m_public_open_space',
    'local_nh_population_density', 'local_nh_intersection_density',
    'local_daily_living', 'local_walkability'
]

fieldNames_new = [
    'pop_pct_access_500m_supermarkets', 'pop_pct_access_500m_convenience',
    'pop_pct_access_500m_pt_any', 'pop_pct_access_500m_public_open_space',
    'pop_nh_pop_density', 'pop_nh_intersection_density', 'pop_daily_living',
    'pop_walkability'
]

In [13]:
gdf_study_region = aggregation(gdf_hex, gdf_study_region,
                               list(zip(fieldNames, fieldNames_new)))

In [14]:
gdf_study_region


,Study region,geometry,urban_sample_point_count,pop_pct_access_500m_supermarkets,pop_pct_access_500m_convenience,pop_pct_access_500m_pt_any,pop_pct_access_500m_public_open_space,pop_nh_pop_density,pop_nh_intersection_density,pop_daily_living,pop_walkability
0,Odense,"MULTIPOLYGON (((589115.578 6145014.850, 589126...",50707,36.870572,36.216761,62.426846,96.633922,2483.269751,121.900978,2.321481,0.376287
